# Lumigator from [Mozilla.ai](https://www.mozilla.ai/) 🐊 🦊

This Notebook will guide you through:

+ What Jupyter Notebooks are
+ An Overview of Machine Learning
+ The Lumigator Platform 🐊
+ How does a Machine Learning evaluation workflows looks like
+ The Thunderbird Ground Truth Dataset
+ Selecting models to perform summarization:
  + Using one encoder/decoder (BART)
  + Utilizing two decoders (Mistral and GPT-4) to evaluate against ground truth
+ Running evaluation experiments
+ Discussing results

## Jupyter Walkthrough

[Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/stable/) provide an executable environment for running (usually) Python code alongside text. To work with Jupyter, click "Run Cell" to execute the code and view the results below the cell you are currently running. Cells are executed sequentially and can contain either text (Markdown) or Python code.

### Running cells 
To run a cell, press the "play" icon in the top bar (you can also hit Shift+Enter to run and proceed to the following cell).

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/running.png" alt="drawing" width="400"/>

Your files are located on the left-hand side. They'll be saved for the duration of our session, but if you'd like to keep them, make sure to download them. 

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/files.png" alt="drawing" width="400"/>

Lets' try running some code! Execute the following code and verify the output below.

In [ ]:
print("Welcome to Lumigator!🐊")

## Machine learning glossary

Some terms you'll hear us using throughout the session: 

+ **Machine learning** - The process of creating a model that learns from data
+ **Dataset** - Data used to train models and evaluate their performance
+ **LLM** - Large language model, [a text-based model that performs next-word predictions](https://www.nvidia.com/en-us/glossary/large-language-models/) 
+ **Tokens** - Words broken up into pieces to be used in an LLM 
+ **Inference** - The process of getting a prediction from a large language model 
+ **Embeddings** - Numerical representations of text generated by modeling 
+ **Encoder-decoder models** - A neural network architecture comprised of two neural networks, an encoder that takes the input vectors from our data and creates an embedding of a fixed length, and a decoder, also a neural network, which takes the embeddings encoded as input and generates a static set of outputs such as translated text or a text summary
+ **Decoder-only models** - Given a fixed input prompt, uses its representation to generate a sequence of words one at a time, with each word being conditioned on the ones generated previously
+ **Task** - Machine learning tasks to fit a specific model type, including translation, summarization, completion, etc. 
+ **Ground truth** - Information that has been evaluated to be true by humans (or LLMs, in some cases), that we can use to evaluate and compare trained models. 

The process of **machine learning** is the process of creating a mathematical model that tries to approximate the world. A **machine learning model** is a set of instructions for generating a given
output from data. The instructions are learned from the features of the input data itself.

Within the universe of modeling approaches, there are **supervised** and **unsupervised** approaches, as well as reinforcement learning. 
Language modeling of the kind we do with LLMs falls in the realm of neural network approaches. 

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/ml_family.png" alt="drawing" width="400"/>

### How do LLMs work? 

There are many different kinds of LLMs and many different kinds of architectures. For our evaluations, we use two different kinds:

+ **Encoder/Decoder** - BART is an encoder/decoder model that converts input data into a fixed-size representation (similar to encoder models). These models are trained first to transform text into numerical representations, then to output text based on those numerical representations. They're good for synthesis as opposed to generation. 
+ **Decoder-only** - most GPT-family models, like Mistral, GPT, and others we'll be working with, are pre-trained with text data in an autoregressive manner, for next-token prediction given previous tokens.  

### LLM Evaluation Workflows

The following steps outline the phases that an LLM goes through during evaluation:

1. Generate ground truth for our business use-case
1. Pick several models we'd like to use to evaluate
1. Run an evaluation loop consisting of looking at the ground truth in comparison to model output
1. Analyze the evaluation results

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/llm_steps.png" alt="drawing" width="600"/>

Lumigator on a technical level is a **Python-based FastAPI web app** with services that run **jobs and deployments on a Ray cluster**, which can be run either locally or in the cloud, depending on your computer specs. Results and job metadata are stored in an SQL database. Larger models loaded from [HuggingFace](https://huggingface.co/) require GPUs.  

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/platform.png" alt="drawing" width="400"/>

What is Ray? [A distributed runtime for Python programs](https://github.com/ray-project/ray) that includes a Core library with primitives (Tasks, Actors, and objects) and a suite of ML libraries (Tune, Serve) that allow to build components of the machine learning model workflow. 

### Nota bene: Machine learning is alchemy

When we think of traditional software application workflows, we think of an example such as adding a button. We can clearly test that we've added a blue button to our application, and that it works correctly. Machine learning is not like this! It involves a lot of experimentation, tweaking of hyperparameters and prompts and trying different models. Expect for the process to be imperfect, with many iterative loops. Luckily, Lumigator helps take away the uncertainty of at least model selection. 🙂

> There’s a self-congratulatory feeling in the air. We say things like “machine learning is the new electricity”. I’d like to offer an alternative metaphor: machine learning has become alchemy. - [Ben Recht and Ali Rahimi](https://archives.argmin.net/2017/12/05/kitchen-sinks/)

Ultimately, the final conclusion of whether a model is good is if humans think it's good. 

With that in mind, let's dive into setting up experiments with Lumigator to test our models!

In [ ]:
# Importing packages we need to work with data 
# python standard libraries
import os
import time
import json

# Random string generator
import random
import string
import shortuuid

# third-party libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from IPython.display import clear_output

from sdk.lumigator import LumigatorClient
from schemas.jobs import JobType, JobCreate
from schemas.datasets import DatasetFormat

# wrap columns for inspection
pd.set_option('display.max_colwidth', 0)
# stylesheet for visibility
plt.style.use("fast")

%load_ext autoreload
%autoreload 2

# Understanding the Lumigator App and API 

The app itself consists of an API, which you can access and test out methods in the [OpenAPI spec](https://swagger.io/specification/), at the platform URL, under docs.
If you are running Lumigator as a local installation, you can directly access the API at [this URL](http://localhost:8000/docs).

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/openapi.png" alt="drawing" width="200"/>

Large language models today are consumed in one of several ways:

+ As **API endpoints** for proprietary models hosted by [OpenAI](https://openai.com/), [Anthropic](https://www.anthropic.com/), or major cloud providers
+ As **model artifacts** downloaded from HuggingFace’s Model Hub and/or trained/fine-tuned using HuggingFace libraries and hosted on local storage
+ As model artifacts available in a format optimized for **local inference**, typically [GGUF](https://github.com/ggerganov/ggml/blob/master/docs/gguf.md), and accessed via applications like [llama.cpp](https://github.com/ggerganov/llama.cpp) or [ollama](https://ollama.com/)
+ As [ONNX](https://onnx.ai/), a format which optimizes sharing between backend ML frameworks

We use API endpoints and local storage in Lumigator. We currently have 5 key endpoints on the platform:

+ `Health` - Status of the application, running status of jobs and deployments. 
+ `Datasets` - Data that we add to the platform for evaluation. We can upload, delete, and save different data in the platform. - We'll use this to save our ground truth and experiment data
+ `Jobs` - Our actual evaluation experiments. We can list all previous evaluations, create new ones, and get their results.
+ `Completions` - Access to external APIs such as Mistral and OpenAI

## Model Task: Summarization

The task we'll be working with is **summarization**, aka we want to generate a summary of our text. 

In our business case, which is to create summaries of conversation threads, much as you might see in Slack or an email chain, the models need to be able to extract key information from those threads while still being able to accept a large context window to capture the entire conversation history. 

We identified that it is far more valuable to conduct **abstractive** summaries—summaries that identify important sections in the text and generate highlights—rather than **extractive** ones, which select a subset of sentences and staple them together. This is because the final interface will be in natural language, and we want to avoid summaries that are interpreted from often incoherent text snippets produced by extractive methods.

For more on summarization as a use-case, [see our blog post here.](https://blog.mozilla.ai/on-model-selection-for-text-summarization/)

## Ground Truth for Models

The term ground truth comes from geology and geospatial sciences, where actual information was collected on the ground to validate data acquired through remote sensing, such as satellite imagery or aerial photography. Since then, the concept has been adopted in other fields, particularly in machine learning and artificial intelligence, to refer to the accurate, real-world data used for training and testing models. 

The **best ground truth is human-generated** but building it is a very expensive task. One recent trend is to rely on large language models but (as you will see later) they have their own pitfalls. An intermediate approach uses different LLMs to provide ground truth "candidates" which are then subject to human pairwise evaluation.

## Our Input data

The data we'll be using in this walkthrough comes from [DialogSum](https://github.com/cylnlp/DialogSum), a large-scale labeled dialogue summarization dataset which comes with ground truth provided by human annotators.

For the sake of explanation, in the next section we will show how Lumigator allows one to generate a ground truth candidates with the help of a language model. However, in the following evaluation section we will compare against the original, human-provided, annotations.

Here follows a brief description of DialogSum.

In [ ]:
# The dataset is available at https://huggingface.co/datasets/knkarthick/dialogsum
# and can be directly downloaded with the `load_dataset` method
dataset = 'knkarthick/dialogsum'
ds = load_dataset(dataset, split='validation')
df = ds.to_pandas()

In [ ]:
# Examine a single sample 
df['dialogue'].iloc[0]

In [ ]:
# Add a function to do some simple character counts for model input
df['char_count'] = df['dialogue'].str.len()

In [ ]:
# inspect our data
df.head(n=3)

In [ ]:
# Show statistics about characters count
df['char_count'].describe()

In [ ]:
# Generate plot of character counts
fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(df['char_count'], bins=30)
ax.set_xlabel('Character Count')
ax.set_ylabel('Frequency')

stats = df['char_count'].describe().apply(lambda x: f"{x:.0f}")

# Add text boxes for statistics
plt.text(1.05, 0.95, stats.to_string(), 
         transform=ax.transAxes, verticalalignment='top')

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(right=0.75)

plt.show()

## Ground Truth Generation with Mistral

In the following we will use [Mistral API](https://docs.mistral.ai/api/) to generate candidate ground truth. Note that for the following code to work, you need to have a valid mistral API key (no worries if you don't, the next section shows another methods which relies on an opensource model running locally on your computer).

In [ ]:
lm_client = LumigatorClient(f"{os.getenv('LUMIGATOR_SERVICE_HOST', 'localhost')}:8000")

In [ ]:
# Perform Ground Truth Generation with Mistral
vendor = "mistral"
mistral_responses = []

for sample in df['dialogue'][0:10]:
    res = lm_client.completions.get_completion(vendor, sample)
    print(f"Mistral Summary:", res)
    mistral_responses.append((sample, res.text))

In [ ]:
# Let's create a result set we can look at
mistral_results_df = pd.DataFrame(mistral_responses, columns=['examples', 'mistral_response'])
mistral_results_df.head()

## Save and upload datasets

Now that you’ve seen how to generate ground truth using an LLM, let’s save the dataset and make it available to Lumigator for further experiments. For each example (the `dialogsum` original dataset and the `mistral-generated` ground truth), we will perform the following operations:

1. Make sure that the two main fields (original text and ground truth) are called `examples` and `ground_truth`, which are the names internally used by Lumigator to refer to them, and save the datasets as CSV files.
1. Make the dataset available to Lumigator with the `create_dataset` method provided by the Lumigator SDK.

In [ ]:
ds = ds.remove_columns(["id", "topic"])
ds = ds.rename_column("dialogue", "examples")
ds = ds.rename_column("summary", "ground_truth")

dataset_name = "dialogsum_converted.csv"
ds.to_csv(dataset_name)

In [ ]:
lm_client.datasets.create_dataset(
    open(dataset_name, "rb"),
    DatasetFormat.JOB
)

In [ ]:
mistral_results_df = mistral_results_df.rename(columns={"mistral_response": "ground_truth"})
mistral_results_df.to_csv('mistral_ground_truth.csv', index=False)

dataset_name = "mistral_ground_truth.csv"

lm_client.datasets.create_dataset(
    open(dataset_name, "rb"),
    DatasetFormat.JOB
)

In [ ]:
# And let's check that data loaded
datasets = lm_client.datasets.get_datasets()
datasets

## Jobs

After generating the ground truth (either manually or with the aid of some models) and uploading the dataset to lumigator, we are ready to start evaluating models on it. Note that when you uploaded your datasets you got back some information that included a dataset `id`. This is a unique identifier to your own dataset that you can reuse across different jobs.

In [ ]:
dataset_id = datasets.items[0].id

# now look for the dataset on lumigator
result = lm_client.datasets.get_dataset(dataset_id)
dataset_id, dataset_name = result.id, result.filename

## Model Selection

What you see below are different lists of models we have already tested for the summarization task.
The `models` variable at the end provides you with a selection, but you can choose any combination of them:
the default is a single local model (`facebook/bart-large-cnn`), but depending on your setup you can choose
more and/or add different APIs.

Note that different model types are specified with different prefixes:

- `hf://` is used for HuggingFace models which are downloaded and ran as Ray jobs
- `mistral://` is used for models which are accessed through the Mistral API
- `oai://` is used for models which are accessed through an OpenAI-compatible API

In [ ]:
# Here follows a list of models we have tested for summarization:
# feel free to add any of them in the "models" list below
#
# Encoder-Decoder models
#    'hf://facebook/bart-large-cnn',
#    'hf://mikeadimech/longformer-qmsum-meeting-summarization', 
#    'hf://mrm8488/t5-base-finetuned-summarize-news',
#    'hf://Falconsai/text_summarization',
#
# Decoder models
#    'mistral://open-mistral-7b',
#
# GPTs
#    "oai://gpt-4o-mini",
#    "oai://gpt-4-turbo",
#    "oai://gpt-3.5-turbo-0125",
#
models = [
    'hf://facebook/bart-large-cnn',
]

## Run Evaluations

The following cell will start the actual model evaluations.
Once you run it, new jobs will be submitted to ray (one for each model) and the outcomes of these submissions will be printed.
Each evaluation job will first use the provided model to summarize each of the emails in the input dataset. After that, it will calculate a few metrics to evaluate how close the predicted summaries are to the ground truth provided in the dataset.

Each job starts with a `created` status. While the job runs, you will be able to track its status by running the cell in the section **Track evaluation jobs**.

In [ ]:
# set this value to limit the evaluation to the first max_samples items (0=all)
max_samples = 10
# team_name is a way to group jobs together under the same namespace, feel free to customize it
team_name = "lumigator_enthusiasts"

responses = []
for model in models:
    job_args = JobCreate(
        name=team_name,
        description="Test",
        model=model,
        dataset=str(dataset_id),
        max_samples=max_samples
    )
    # descr = f"Testing {model} summarization model on {dataset_name}"
    responses.append(lm_client.jobs.create_job(JobType.EVALUATION, job_args))

### Track evaluation jobs

Run the following to track your evaluation jobs.

- *NOTE*: you won't be able to run other cells while this one is running. However, you can interrupt it whenever you want by clicking on the "stop" button above and run it at a later time.

In [ ]:
job_id = responses[0].id

job = lm_client.jobs.wait_for_job(job_id)  # Create the coroutine object
result = await job         # Await the coroutine to get the result
print(result)

## Show evaluation results

Once all evaluations are completed, their results will be stored on our platform and available for download. You can download them individually using the `get_job_result()` method.

The following cell iterates on all your job ids, downloads results from each, and builds a table comparing different metrics for each model.
The metrics we use to evaluate are ROUGE, METEOR, and BERT score. They all measure similarity between predicted summaries and those provided with the ground truth, but each of them focuses on different aspects. The image below shows their main characteristics and the tradeoffs between their flexibility and their computational cost.

<img src="https://raw.githubusercontent.com/mozilla-ai/lumigator/main/docs/assets/metrics.png" alt="drawing" width="900"/>

In [ ]:
job_ids = [job.id for job in lm_client.jobs.get_jobs().items]

In [ ]:
# after the jobs complete, gather evaluation results
eval_results = []
for job_id in job_ids:
    eval_results.append(lm_client.jobs.get_job_download(job_id))

In [ ]:
eval_results[0].download_url

In [ ]:
responses = []
for eval_result in eval_results:
    response = requests.request(url=eval_result.download_url, method="GET")
    responses.append(response.text)

## Analysis of Evaluation Results

The tablel above is just a summary of all the evaluation results.
The `eval_results` object contains way more details from which you'll be able to get a few more insights in the following cells.

### Direct access to all data

The following cell shows you the kind of information that's available in each of the `eval_results` elements. This information is nested at different depth levels. You can access each using the `get_nested_value` command.

In [ ]:
# eval_results is a list holding information for each of the models you defined before
# for each element, you can access different metrics, time performance, and predictions
eval_results[0].keys()

In [ ]:
# see how much time it took for a model to summarize all the input samples
ld.get_nested_value(eval_results[0], "summarization_time")

In [ ]:
# see all the bertscore data
ld.get_nested_value(eval_results[0], "bertscore")

In [ ]:
# see mean bert precision
ld.get_nested_value(eval_results[0], "bertscore/precision_mean")